<a href="https://colab.research.google.com/github/ayman-06-stack/BIGDATALABS/blob/main/Lab_spark_sql-FIFA/lab_spark_sql_fifa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max, count, desc, sum, avg, when, lit
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit, when, floor, abs, desc, sum, avg
from pyspark.sql.functions import year
from pyspark.sql.functions import col, lit, year, sum, count, desc, row_number, rank, lag, when, max

In [3]:
spark = SparkSession.builder.appName("FIFA_World_Cup_Analysis").getOrCreate()

In [5]:
df = spark.read.csv("fifaworldcup (1).csv", header=True, inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: string (nullable = true)
 |-- away_score: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)



In [13]:
df.count()

14536

In [14]:
df.show(5)

+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  false|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  false|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  false|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 5 rows


In [15]:
df.select(min("date"), max("date")).show()

+----------+----------+
| min(date)| max(date)|
+----------+----------+
|1872-11-30|1986-06-17|
+----------+----------+



In [16]:
df.groupBy("tournament").count().orderBy(desc("count"))

DataFrame[tournament: string, count: bigint]

In [17]:
df.filter(col("neutral") == True).count()

3009

In [18]:
df.groupBy("country")

GroupedData[grouping expressions: [country], value: [date: date, home_team: string ... 7 more fields], type: GroupBy]

In [19]:
df.filter((col("home_score") + col("away_score")) > 6).show()

+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|      date|       home_team|       away_team|home_score|away_score|          tournament|     city| country|neutral|
+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|1878-03-02|        Scotland|         England|         7|         2|            Friendly|  Glasgow|Scotland|  false|
|1878-03-23|        Scotland|           Wales|         9|         0|            Friendly|  Glasgow|Scotland|  false|
|1879-04-05|         England|        Scotland|         5|         4|            Friendly|   London| England|  false|
|1880-03-13|        Scotland|         England|         5|         4|            Friendly|  Glasgow|Scotland|  false|
|1881-03-12|         England|        Scotland|         1|         6|            Friendly|   London| England|  false|
|1882-02-18|Northern Ireland|         England|         0|       

In [7]:
df_home = df.select(
    col("date"),
    col("home_team").alias("team"),
    col("home_score").alias("goals_for"),
    col("away_score").alias("goals_against"),
    lit("home").alias("venue")
)

In [8]:
df_away = df.select(
    col("date"),
    col("away_team").alias("team"),
    col("away_score").alias("goals_for"),
    col("home_score").alias("goals_against"),
    lit("away").alias("venue")
)

In [9]:
df_teams = df_home.union(df_away)

In [10]:
q8_result = df_teams.groupBy("team") \
    .count() \
    .withColumnRenamed("count", "total_matches") \
    .orderBy(desc("total_matches"))

In [11]:
q8_result.show(10)

+-----------+-------------+
|       team|total_matches|
+-----------+-------------+
|    England|          842|
|     Sweden|          834|
|    Hungary|          796|
|  Argentina|          795|
|     Brazil|          790|
|    Germany|          757|
|    Uruguay|          717|
|     Norway|          665|
|   Scotland|          664|
|South Korea|          662|
+-----------+-------------+
only showing top 10 rows


In [12]:
q9_result = df_teams.groupBy("team") \
    .agg(sum("goals_for").alias("total_goals")) \
    .orderBy(desc("total_goals")) \
    .limit(10)

In [13]:
q9_result.show()

+-----------+-----------+
|       team|total_goals|
+-----------+-----------+
|    England|       1871|
|     Brazil|       1745|
|     Sweden|       1743|
|    Hungary|       1716|
|    Germany|       1661|
|  Argentina|       1482|
|Netherlands|       1271|
|South Korea|       1211|
|    Austria|       1211|
|   Scotland|       1197|
+-----------+-----------+



In [15]:
df_decades = df.withColumn("decade", year("date") - (year("date") % 10)) \
               .withColumn("match_goals", col("home_score") + col("away_score"))

In [16]:
q10_result = df_decades.groupBy("decade") \
    .agg(avg("match_goals").alias("avg_goals_per_match")) \
    .orderBy("decade")

In [17]:
q10_result.show()

+------+-------------------+
|decade|avg_goals_per_match|
+------+-------------------+
|  1870|  4.538461538461538|
|  1880|  5.581818181818182|
|  1890| 5.1525423728813555|
|  1900|  4.169354838709677|
|  1910|  4.007067137809187|
|  1920| 3.8367626886145403|
|  1930|   4.25940594059406|
|  1940|  4.294776119402985|
|  1950|  3.980964467005076|
|  1960| 3.4683274021352313|
|  1970|  2.955379908210097|
|  1980| 2.4950990615224193|
|  1990|  2.730442692540934|
|  2000|  2.856734036844929|
+------+-------------------+



In [18]:
q11_result = df.withColumn("year", year("date")) \
    .groupBy("tournament", "year") \
    .count() \
    .orderBy(desc("count"))

In [19]:
q11_result.show(10)

+--------------------+----+-----+
|          tournament|year|count|
+--------------------+----+-----+
|FIFA World Cup qu...|2001|  489|
|FIFA World Cup qu...|2004|  412|
|FIFA World Cup qu...|1997|  408|
|            Friendly|2004|  379|
|            Friendly|2000|  376|
|            Friendly|2002|  358|
|FIFA World Cup qu...|2005|  353|
|FIFA World Cup qu...|1993|  335|
|            Friendly|2003|  324|
|            Friendly|1996|  306|
+--------------------+----+-----+
only showing top 10 rows


In [20]:
q12_result = df.filter(col("home_score") > col("away_score")) \
    .groupBy("home_team") \
    .count() \
    .withColumnRenamed("count", "home_wins") \
    .orderBy(desc("home_wins"))

In [21]:
q12_result.show(10)

+-----------+---------+
|  home_team|home_wins|
+-----------+---------+
|     Brazil|      331|
|  Argentina|      293|
|    England|      245|
|    Germany|      244|
|     Sweden|      234|
|      Italy|      224|
|    Hungary|      224|
|South Korea|      213|
|     Mexico|      209|
|     France|      205|
+-----------+---------+
only showing top 10 rows


In [22]:
df_wld = df_teams.withColumn("win", when(col("goals_for") > col("goals_against"), 1).otherwise(0)) \
                 .withColumn("draw", when(col("goals_for") == col("goals_against"), 1).otherwise(0)) \
                 .withColumn("loss", when(col("goals_for") < col("goals_against"), 1).otherwise(0))

In [23]:
q13_result = df_wld.groupBy("team") \
    .agg(
        sum("win").alias("total_wins"),
        sum("draw").alias("total_draws"),
        sum("loss").alias("total_losses")
    ) \
    .orderBy(desc("total_wins"))

In [24]:
q13_result.show(10)

+-----------+----------+-----------+------------+
|       team|total_wins|total_draws|total_losses|
+-----------+----------+-----------+------------+
|     Brazil|       493|        162|         135|
|    England|       471|        204|         167|
|    Germany|       431|        158|         168|
|  Argentina|       419|        199|         177|
|     Sweden|       412|        184|         238|
|    Hungary|       379|        178|         239|
|South Korea|       353|        174|         135|
|      Italy|       333|        168|         120|
|   Scotland|       318|        148|         198|
|     Mexico|       307|        157|         169|
+-----------+----------+-----------+------------+
only showing top 10 rows


In [25]:
df_neutral = df.withColumn("total_goals", col("home_score") + col("away_score"))

In [26]:
q14_result = df_neutral.groupBy("neutral") \
    .agg(avg("total_goals").alias("avg_goals"))

In [27]:
q14_result.show()

+-------+-----------------+
|neutral|        avg_goals|
+-------+-----------------+
|   true|3.174299065420561|
|  false|3.009568505946143|
+-------+-----------------+



In [28]:
q15_result = df.withColumn("score_diff", abs(col("home_score") - col("away_score"))) \
    .select("date", "home_team", "home_score", "away_team", "away_score", "score_diff") \
    .orderBy(desc("score_diff")) \
    .limit(5)

In [29]:
q15_result.show(truncate=False)

+----------+---------+----------+--------------+----------+----------+
|date      |home_team|home_score|away_team     |away_score|score_diff|
+----------+---------+----------+--------------+----------+----------+
|2001-04-11|Australia|31        |American Samoa|0         |31        |
|1971-09-13|Tahiti   |30        |Cook Islands  |0         |30        |
|1979-08-30|Fiji     |24        |Kiribati      |0         |24        |
|2001-04-09|Australia|22        |Tonga         |0         |22        |
|1966-04-03|Libya    |21        |Oman          |0         |21        |
+----------+---------+----------+--------------+----------+----------+



In [31]:
df_home = df.select(
    col("date"),
    col("tournament"),
    col("home_team").alias("team"),
    col("home_score").alias("goals_for"),
    col("away_score").alias("goals_against"),
    lit("home").alias("venue")
)

In [32]:
df_away = df.select(
    col("date"),
    col("tournament"),
    col("away_team").alias("team"),
    col("away_score").alias("goals_for"),
    col("home_score").alias("goals_against"),
    lit("away").alias("venue")
)

In [33]:
df_teams = df_home.union(df_away).withColumn("year", year("date"))

In [34]:
df_teams = df_teams.withColumn("is_win", when(col("goals_for") > col("goals_against"), 1).otherwise(0))

In [35]:
q16_result = df_teams.groupBy("team") \
    .agg(
        (sum("goals_for") - sum("goals_against")).alias("goal_average")
    ) \
    .orderBy(desc("goal_average"))

In [36]:
q16_result.show(10)

+-----------+------------+
|       team|goal_average|
+-----------+------------+
|    England|        1008|
|     Brazil|         973|
|    Germany|         753|
|  Argentina|         627|
|South Korea|         623|
|     Sweden|         618|
|    Hungary|         499|
|      Spain|         476|
|      Italy|         467|
|Netherlands|         451|
+-----------+------------+
only showing top 10 rows


In [37]:
wins_per_year = df_teams.groupBy("year", "team") \
    .agg(sum("is_win").alias("total_wins"))

In [38]:
windowSpec = Window.partitionBy("year").orderBy(desc("total_wins"))

In [39]:
q17_result = wins_per_year.withColumn("rank", row_number().over(windowSpec)) \
    .filter(col("rank") == 1) \
    .orderBy(desc("year"))

In [40]:
q17_result.show(10)

+----+-------------+----------+----+
|year|         team|total_wins|rank|
+----+-------------+----------+----+
|2005|United States|        13|   1|
|2004|        Japan|        16|   1|
|2003|       France|        13|   1|
|2002|       Brazil|        14|   1|
|2001| Saudi Arabia|        17|   1|
|2000|         Iran|        15|   1|
|1999|       Brazil|        16|   1|
|1998| Saudi Arabia|        11|   1|
|1997|       Brazil|        20|   1|
|1996| Saudi Arabia|        14|   1|
+----+-------------+----------+----+
only showing top 10 rows


In [41]:
df_decades = df.withColumn("decade", year("date") - (year("date") % 10))

In [42]:
q18_result = df_decades.groupBy("decade") \
    .count() \
    .orderBy("decade")

In [43]:
q18_result.show()

+------+-----+
|decade|count|
+------+-----+
|  1870|   13|
|  1880|   55|
|  1890|   59|
|  1900|  124|
|  1910|  283|
|  1920|  729|
|  1930| 1010|
|  1940|  804|
|  1950| 1576|
|  1960| 2810|
|  1970| 3922|
|  1980| 4795|
|  1990| 6596|
|  2000| 5591|
+------+-----+



In [44]:
df_losses = df_teams.withColumn("is_loss", when(col("goals_for") < col("goals_against"), 1).otherwise(0))

In [45]:
q19_result = df_losses.groupBy("year", "team") \
    .agg(
        sum("is_loss").alias("total_losses"),
        count("*").alias("matches_played")
    ) \
    .filter((col("total_losses") == 0) & (col("matches_played") > 5)) \
    .orderBy(desc("year"), desc("matches_played"))

In [46]:
q19_result.show(10)

+----+---------+------------+--------------+
|year|     team|total_losses|matches_played|
+----+---------+------------+--------------+
|2005|    Italy|           0|            13|
|2005|    Spain|           0|            12|
|2005|   France|           0|            11|
|2005|    Ghana|           0|             8|
|2005| Cameroon|           0|             7|
|2005|Singapore|           0|             6|
|2004|     Cuba|           0|            10|
|2004| Ethiopia|           0|             6|
|2003|   Greece|           0|            11|
|2003| DR Congo|           0|            10|
+----+---------+------------+--------------+
only showing top 10 rows


In [47]:
w_team_date = Window.partitionBy("team").orderBy("date")

In [48]:
df_streak = df_teams.withColumn("not_win", when(col("is_win") == 1, 0).otherwise(1))

In [49]:
df_streak = df_streak.withColumn("streak_id", sum("not_win").over(w_team_date))

In [50]:
streak_counts = df_streak.filter(col("is_win") == 1) \
    .groupBy("team", "streak_id") \
    .count() \
    .withColumnRenamed("count", "streak_length")

In [51]:
q20_result = streak_counts.groupBy("team") \
    .agg(max("streak_length").alias("max_winning_streak")) \
    .orderBy(desc("max_winning_streak"))

In [52]:
q20_result.show(10)

+---------+------------------+
|     team|max_winning_streak|
+---------+------------------+
|Mauritius|                17|
|   Brazil|                14|
|   France|                14|
|Australia|                13|
|   Mexico|                13|
| Scotland|                13|
|Indonesia|                12|
|  Germany|                12|
|German DR|                12|
|   Russia|                12|
+---------+------------------+
only showing top 10 rows


In [53]:
wins_per_tourney = df_teams.groupBy("tournament", "team") \
    .agg(sum("is_win").alias("total_wins"))

In [54]:
w_tourney = Window.partitionBy("tournament").orderBy(desc("total_wins"))

In [55]:
q21_result = wins_per_tourney.withColumn("rank", rank().over(w_tourney)) \
    .filter(col("rank") == 1) \
    .orderBy(desc("total_wins"))

In [56]:
q21_result.show(10)

+--------------------+-----------+----------+----+
|          tournament|       team|total_wins|rank|
+--------------------+-----------+----------+----+
|            Friendly|    Germany|       269|   1|
|British Championship|    England|       151|   1|
|        Copa América|  Argentina|       105|   1|
| Nordic Championship|     Sweden|        83|   1|
|  Merdeka Tournament|   Malaysia|        77|   1|
|FIFA World Cup qu...|     Sweden|        62|   1|
|      FIFA World Cup|     Brazil|        60|   1|
|UEFA Euro qualifi...|      Spain|        55|   1|
|          CECAFA Cup|     Uganda|        54|   1|
|           Korea Cup|South Korea|        51|   1|
+--------------------+-----------+----------+----+
only showing top 10 rows


In [57]:
q22_raw = df_teams.groupBy("team", "venue") \
    .agg(avg("is_win").alias("win_rate"))

In [58]:
q22_result = q22_raw.groupBy("team") \
    .pivot("venue") \
    .sum("win_rate") \
    .orderBy(desc("home"))

In [59]:
q22_result.show(10)

+-----------------+-------------------+------------------+
|             team|               away|              home|
+-----------------+-------------------+------------------+
|    Central Spain|                0.0|               1.0|
|Western Australia| 0.3225806451612903|               1.0|
|   Czech Republic|0.44155844155844154|               0.8|
|           Jersey|0.36363636363636365|0.7931034482758621|
|     Saint Martin|              0.125|              0.75|
|           Brazil| 0.4954128440366973| 0.714902807775378|
|   Basque Country| 0.6842105263157895|0.7142857142857143|
|         Shetland| 0.1111111111111111|            0.6875|
|   Canary Islands|               NULL|0.6666666666666666|
|    Western Isles|                0.0|0.6666666666666666|
+-----------------+-------------------+------------------+
only showing top 10 rows
